In [3]:
import mesa
from mesa import Model, Agent
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from market_model import DataMarket, Participants
from mesa.batchrunner import batch_run
from analysis_plots import plot_final_holders_by_persona


In [4]:
persona_dist = [{
    'Bandwagon': 0.7,
    'Neutral': 0.15,
    'Snob': 0.15
},{
    'Bandwagon': 0.15,
    'Neutral': 0.15,
    'Snob': 0.7
},{
    'Bandwagon': 0.15,
    'Neutral': 0.7,
    'Snob': 0.15
}]


w1_params = [(2.0, 100)]

In [5]:
# test_model = DataMarket(num_agents = 1000, num_data = 1000, initial_price = 200, persona_dist = persona_dist, wealth_alpha = 2, wealth_scale = 100, w1_params = (2, 100), tau=0.1, seed=None)

# for _ in range(50):
#     test_model.step()
#     test_model.datacollector.collect(test_model)



In [6]:

# 全体レンジ（10刻み）
base_prices = np.logspace(
    np.log10(10),     # 10 の log10
    np.log10(2000),   # 2000 の log10
    num=40            # 点の数（元の 1〜40 と同じ）
)

# 必要なら四捨五入して整数に
base_prices = base_prices.round().astype(int)

print(base_prices)

all_results = []

for dist in persona_dist:
    params = {
        "num_agents":1000,
        "num_data": 1000,
        "initial_price": base_prices,
        "tau": 0.2,
        "persona_dist": [dist],
        "wealth_alpha": 2.0,
        "wealth_scale": 100,
        "w1_params": w1_params,
        "seed": 42,
        "dynamic_pricing": False
    }
    results = batch_run(
        model_cls=DataMarket,
        parameters=params,
        iterations=3,           # 各価格につき 3回繰り返す
        max_steps=50,           # 各シミュレーションのステップ数上限
        number_processes=None,   # CPU 全て使いたいなら None
        data_collection_period=1,
        display_progress=True,
    )


    # dist の種類を記録したい場合はここで付与しておく
    for r in results:
        r['persona_dist'] = str(dist)   # or dist.copy()

    all_results.extend(results)  # ← 上書きしない
    
results_df = pd.DataFrame(all_results)

results_df.to_csv("results_certain.csv", index=False, encoding="utf-8")


[  10   11   13   15   17   20   23   26   30   34   39   45   51   58
   67   77   88  101  115  132  151  173  199  228  261  299  342  392
  449  514  589  675  773  885 1014 1162 1331 1524 1746 2000]


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

In [7]:
print(results_df.columns)

results_df.head()

Index(['RunId', 'iteration', 'Step', 'num_agents', 'num_data', 'initial_price',
       'tau', 'persona_dist', 'wealth_alpha', 'wealth_scale', 'w1_params',
       'seed', 'dynamic_pricing', 'Holders', 'ProviderRevenue', 'CurrentPrice',
       'AgentID', 'w_1', 'w_n', 'alpha', 'lamb', 'l_n_minus_1', 'HasToken',
       'Wealth', 'Persona', 'BoughtStep', 'CurrentUtility'],
      dtype='object')


,RunId,iteration,Step,num_agents,num_data,initial_price,tau,persona_dist,wealth_alpha,wealth_scale,...,w_1,w_n,alpha,lamb,l_n_minus_1,HasToken,Wealth,Persona,BoughtStep,CurrentUtility
0,2,0,0,1000,1000,13,0.2,"{'Bandwagon': 0.7, 'Neutral': 0.15, 'Snob': 0.15}",2.0,100,...,161.560571,68.327925,0.422925,0.450548,-72.790849,False,110.701367,Neutral,NaN,0.0
1,2,0,0,1000,1000,13,0.2,"{'Bandwagon': 0.7, 'Neutral': 0.15, 'Snob': 0.15}",2.0,100,...,127.105289,98.416955,0.774295,0.827710,-105.206300,False,116.417781,Bandwagon,NaN,0.0
2,2,0,0,1000,1000,13,0.2,"{'Bandwagon': 0.7, 'Neutral': 0.15, 'Snob': 0.15}",2.0,100,...,135.092368,115.464589,0.854708,0.741876,-100.221849,False,103.033317,Bandwagon,NaN,0.0
3,2,0,0,1000,1000,13,0.2,"{'Bandwagon': 0.7, 'Neutral': 0.15, 'Snob': 0.15}",2.0,100,...,279.721816,152.840251,0.546401,0.429240,-120.067739,False,151.761412,Neutral,NaN,0.0
4,2,0,0,1000,1000,13,0.2,"{'Bandwagon': 0.7, 'Neutral': 0.15, 'Snob': 0.15}",2.0,100,...,105.851052,79.694683,0.752895,0.726787,-76.931167,False,100.411395,Bandwagon,NaN,0.0
